<a href="https://colab.research.google.com/github/Akshayvm98/Akshayvm98.github.io/blob/master/Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 16, 3) #input - 28 OUtput 26 RF
        self.batch_norm1 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout2d(0.10)

        self.conv2 = nn.Conv2d(16, 16, 3) # 26 24
        self.batch_norm2 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout2d(0.10)

        self.conv3 = nn.Conv2d(16, 16, 3) # 24 22
        self.batch_norm3 = nn.BatchNorm2d(16)
        #self.dropout3 = nn.Dropout2d(0.15)


        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(16, 16, 3) # 11 9
        self.batch_norm4 = nn.BatchNorm2d(16)
        self.dropout4 = nn.Dropout2d(0.10)

        #self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(16 ,16, 3) # 9 7
        self.batch_norm5 = nn.BatchNorm2d(16)
        self.dropout5 = nn.Dropout2d(0.10)

        self.conv6 = nn.Conv2d(16, 16, 3) # 7 5
        self.batch_norm6 = nn.BatchNorm2d(16)
        self.dropout6 = nn.Dropout2d(0.10)

        self.conv7 = nn.Conv2d(16, 10, 3) # 5 3

        self.gap = nn.AdaptiveAvgPool2d(1)
      


      

      #  self.conv8 = nn.Conv2d(16, 10, 3) # 3 1

    def forward(self, x):
        x = self.dropout1(self.batch_norm1(F.relu(self.conv1(x))))
        x = self.dropout2(self.batch_norm2(F.relu(self.conv2(x))))
        x = self.pool1(self.batch_norm3(F.relu(self.conv3(x))))
        x = self.dropout4(self.batch_norm4(F.relu(self.conv4(x))))
        x = self.dropout5(self.batch_norm5(F.relu(self.conv5(x))))
        x = self.dropout6(self.batch_norm6(F.relu(self.conv6(x))))
      # x = self.dropout7(self.batch_norm7(F.relu(self.conv7(x))))

        x = (self.conv7(x))
        x = self.gap(x)
      
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
         MaxPool2d-9           [-1, 16, 11, 11]               0
           Conv2d-10             [-1, 16, 9, 9]           2,320
      BatchNorm2d-11             [-1, 16, 9, 9]              32
        Dropout2d-12             [-1, 16, 9, 9]               0
           Conv2d-13             [-1, 16, 7, 7]           2,320
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1,16):
    print(f'epoch:{epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

epoch:1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0510, Accuracy: 9839/10000 (98.39%)

epoch:2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0195, Accuracy: 9937/10000 (99.37%)

epoch:14


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0188, Accuracy: 9940/10000 (99.40%)

epoch:15


loss=0.032550591975450516 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 38.52it/s]



Test set: Average loss: 0.0184, Accuracy: 9942/10000 (99.42%)

